In [ ]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath


In [ ]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

SUM_GAIN = (
    2.63652827e-11, 1.47275591e-11, 2.90485928e-11, 1.97866364e-11
)

D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm

In [ ]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [ ]:
#usage
powers = [0,2,4]
no_of_users = 4
num_random_solutions = 8
pop_size_dbm = generate_random_solutions(powers, no_of_users, num_random_solutions)
print(pop_size_dbm)

In [ ]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [ ]:
#usage
linear_pop=calculate_linear_trans_power(pop_size_dbm)
print(linear_pop)

In [ ]:
#usage for converting the noise to linear
linear_noise = calculate_linear_trans_power([N])
print("Linear noise:", linear_noise)

In [ ]:
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [ ]:
#Usage for calculating the bitrates through big formula
bitrate=calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise)
print(bitrate)

In [ ]:
def row_wise_sums(matrix):
    return [sum(row) for row in matrix]

In [ ]:
# USAGE
row_sum_bitrate = row_wise_sums(bitrate)
print("ROW SUMS OF BITRATE:")
print(*row_sum_bitrate, sep='\n')


In [ ]:
def get_sorted_row_sums(matrix):
    # Calculate the sum of each row and store it as a tuple (row_sum, row)
    row_sums = [(sum(row), row) for row in matrix]

    # Sort the row sums in increasing order
    sorted_row_sums = sorted(row_sums, key=lambda x: x[0])

    # Return the sorted row sums
    return [row_sum for row_sum, _ in sorted_row_sums]

In [ ]:
sorted_row_sums = get_sorted_row_sums(bitrate)
print("Row sums in increasing order:")
print(*sorted_row_sums, sep='\n')

In [ ]:
target_bitrate = sorted_row_sums[0]
print(target_bitrate)

In [ ]:
def select_half_of_sorted_sums(sorted_row_sums):
    half_length = len(sorted_row_sums) // 2
    return sorted_row_sums[:half_length]

half_sorted_row_sums = select_half_of_sorted_sums(sorted_row_sums)

print("Half of the sorted row sums:")
for row_sum in half_sorted_row_sums:
    print(row_sum)

In [ ]:
def get_initial_population(bitrate, pop_size_dbm, sorted_row_sums):
    half_sorted_row_sums = select_half_of_sorted_sums(sorted_row_sums)

    initial_population = []
    for sorted_sum in half_sorted_row_sums:
        index = row_wise_sums(bitrate).index(sorted_sum)
        initial_population.append(pop_size_dbm[index])

    return initial_population, half_sorted_row_sums

In [ ]:
# Call the function and get the initial population and their total bitrates
initial_population, initial_population_bitrates = get_initial_population(bitrate, pop_size_dbm, sorted_row_sums)

# Print the initial population and their total bitrates
print("Initial population and their total bitrates:")
for solution, bitrate in zip(initial_population, initial_population_bitrates):
    print("Solution:", solution, "Total bitrate:", bitrate)

In [ ]:
def single_point_crossover(parent1, parent2):
    # Perform single-point crossover
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.hstack((parent2[:crossover_point], parent1[crossover_point:]))
    
    return child1, child2


offspring = []
for i in range(0, len(initial_population), 2):
    parent1 = initial_population[i]
    parent2 = initial_population[i + 1]
    
    child1, child2 = single_point_crossover(parent1, parent2)
    
    offspring.append(child1)
    offspring.append(child2)

print("Initial population:")
for solution in initial_population:
    print(solution)

print("\nOffspring after single-point crossover:")
for child in offspring:
    print(child)


In [ ]:
def mutate_one_gene_per_individual(population, powers):
    mutated_population = np.copy(population)

    for i, individual in enumerate(mutated_population):
        # Choose a random gene within the individual
        gene_index = random.randint(0, len(individual) - 1)

        # Store the original power level of the chosen gene
        original_power = individual[gene_index]

        # Mutate the chosen gene with a random power level from the powers list
        individual[gene_index] = random.choice(powers)

        # Print the mutation information
#         print(f"Offspring {i + 1}: Gene {gene_index + 1} was mutated from {original_power} to {individual[gene_index]}")

    return mutated_population


In [ ]:
mutated_offspring = mutate_one_gene_per_individual(offspring, powers)

print("\nMutated offspring:")
for child in mutated_offspring:
    print(child)
